In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from scipy.stats import spearmanr, kendalltau

In [2]:
x_train = pd.read_csv('data/x_train.csv')[['player_id', 'tournament_id','question_id']]
y_train = pd.read_csv('data/y_train.csv')

/home/maxima1ist/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
y_train_arr = []

for value in y_train['hit']:
    if isinstance(value, str) and value.isdigit():
        y_train_arr.append(int(value))
    else:
        y_train_arr.append(np.random.randint(0, 1))

In [4]:
x_test = pd.read_csv('data/x_test.csv')[['player_id', 'tournament_id','question_id']]
y_test = pd.read_csv('data/y_test.csv')

In [5]:
y_test_arr = []

for value in y_test['hit']:
    if isinstance(value, str) and value.isdigit():
        y_test_arr.append(int(value))
    else:
        y_test_arr.append(np.random.randint(0, 1))

In [6]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(x_train)
enc.categories_

[array([4772, 4973, 4974, 4975, 4986, 5000, 5008, 5009, 5010, 5011, 5012,
        5013, 5021, 5025, 5042, 5052, 5053, 5055, 5056, 5060, 5061, 5065,
        5070, 5071, 5074, 5078, 5083, 5097, 5098, 5103, 5108, 5109, 5110,
        5111, 5112, 5119, 5128, 5129, 5130, 5141, 5154, 5155, 5156, 5159,
        5161, 5162, 5163, 5179, 5185, 5186, 5187, 5188, 5204, 5205, 5206,
        5217, 5218, 5251, 5266, 5267, 5274, 5275, 5276, 5280, 5284, 5285,
        5298, 5303, 5306, 5307, 5310, 5312, 5313, 5314, 5320, 5322, 5324,
        5325, 5331, 5332, 5333, 5334, 5338, 5343, 5350, 5354, 5358, 5360,
        5361, 5362, 5363, 5364, 5366, 5367, 5368, 5369, 5370, 5371, 5372,
        5374, 5377, 5380, 5381, 5383, 5384, 5385, 5386, 5387, 5388, 5389,
        5390, 5391, 5392, 5393, 5394, 5395, 5397, 5398, 5399, 5400, 5401,
        5402, 5404, 5405, 5406, 5407, 5408, 5409, 5410, 5411, 5412, 5413,
        5415, 5416, 5417, 5418, 5419, 5420, 5421, 5422, 5423, 5424, 5425,
        5426, 5427, 5429, 5430, 5431, 

In [7]:
model = LogisticRegression(solver='liblinear', C=10, n_jobs=-1)
model.fit(enc.transform(x_train), y_train_arr)

/home/maxima1ist/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1353: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn("'n_jobs' > 1 does not have any effect when"


LogisticRegression(C=10, n_jobs=-1, solver='liblinear')

In [8]:
y_test_predict_proba = [probas[0] for probas in model.predict_proba(enc.transform(x_test))]

In [9]:
valid_test_results = {}
with open('data/valid_test_results.json') as fout:
    valid_test_results = json.load(fout)

In [10]:
list_of_real_tournament_range = []
list_of_predicted_tournament_range = []

curr_i = 0
for tournament_id in valid_test_results:
    tournament = valid_test_results[tournament_id]

    real_tournament_range = []
    predicted_tournament_range = []
    for team in tournament:
        real_mask = team['mask']
        real_count = 0
        for i in range(len(real_mask)):
            if isinstance(real_mask[i], str) and real_mask[i].isdigit():
                real_count += int(real_mask[i])

        predict_proba = [1 for _ in range(len(real_mask))]
        for teammate in team['teamMembers']:
            for question_id in range(len(real_mask)):
                predict_proba[question_id] *= y_test_predict_proba[curr_i]
                curr_i += 1
        predict_proba = [(1 - p) for p in predict_proba]

        real_tournament_range.append((team['team']['id'], real_count / len(real_mask)))
        predicted_tournament_range.append((team['team']['id'], np.prod(predict_proba)))

    list_of_real_tournament_range.append(real_tournament_range)
    list_of_predicted_tournament_range.append(predicted_tournament_range)

In [11]:
spearmanr_coefs = []
kendalltau_coefs = []

for i in range(len(list_of_real_tournament_range)):
    predict_tournament_rating = list_of_predicted_tournament_range[i]
    predict_tournament_rating.sort(key=lambda x:x[1])
    predict_tournament_rating = [pair[0] for pair in predict_tournament_rating]

    real_tournament_rating = list_of_real_tournament_range[i]
    real_tournament_rating.sort(key=lambda x:x[1])
    real_tournament_rating = [pair[0] for pair in real_tournament_rating]
    coef, p = spearmanr(real_tournament_rating, predict_tournament_rating)
    spearmanr_coefs.append(abs(coef))
    coef, p = kendalltau(real_tournament_rating, predict_tournament_rating)
    kendalltau_coefs.append(abs(coef))

In [12]:
spearmanr_coefs

[0.039347749025168376,
 0.04197466766809833,
 0.12942486085343227,
 0.286377103704578,
 0.08242066991536708,
 0.336468646061096,
 0.14556160124368442,
 0.22452578280474594,
 0.04628326358936055,
 0.16748271889400923,
 0.16059258092876172,
 0.15089063481892614,
 0.13995765254395098,
 0.20575164248772992,
 0.17052201598493635,
 0.16284358467592916,
 0.27696078431372556,
 0.1360518129652753,
 0.09690058227588445,
 0.184921968787515,
 0.027127231211884154,
 0.009489324235086947,
 0.02460162147050601,
 0.020510051005100505,
 0.2116791299823507,
 0.1853637838108479,
 0.20245974306658443,
 0.5191304347826087,
 0.1285614374642908,
 0.19591397849462364,
 0.07273531066634514,
 0.07741761453041406,
 0.08155342898323896,
 0.18171866447728516,
 0.21654135338345865,
 0.09897209255243004,
 0.4727272727272727,
 0.056105953905249674,
 0.17757954463143152,
 0.14749111999112,
 0.16986062717770037,
 0.16385619367330165,
 0.32867132867132864,
 0.36672140120415986,
 0.04175353365338638,
 0.26580086580086587

In [13]:
kendalltau_coefs

[0.014333492594362158,
 0.029651613593219436,
 0.08596165739022883,
 0.19078833879592896,
 0.05525929360847051,
 0.2211399454911719,
 0.10053115688560693,
 0.15220125786163524,
 0.02470273353516739,
 0.10701484895033281,
 0.1112225006961849,
 0.1022849712056474,
 0.09419205329985998,
 0.14267204498227604,
 0.11933276304533791,
 0.10930731991409749,
 0.2205882352941176,
 0.09053688494626276,
 0.06474860713238445,
 0.12653061224489798,
 0.01753993133303478,
 0.006493506493506493,
 0.017239772621377318,
 0.013333333333333334,
 0.14604693552061976,
 0.12543668841869357,
 0.13376289158491517,
 0.34057971014492755,
 0.08630552417196885,
 0.13548387096774192,
 0.047785547785547784,
 0.05422929560860594,
 0.05405462406411473,
 0.12698412698412698,
 0.1368421052631579,
 0.06501495569727413,
 0.3523809523809524,
 0.03961420911396628,
 0.12057045818544719,
 0.09662189662189663,
 0.12682926829268293,
 0.11066475370636057,
 0.1818181818181818,
 0.2751322751322751,
 0.021249737008205344,
 0.17171717

In [14]:
players_dict = {}
with open('data/players.pkl', 'rb') as f:
    players_dict = pickle.load(f)

players_df = pd.DataFrame(players_dict).transpose().rename(columns = {'id': 'player_id'}, inplace = False)

In [15]:
players = np.unique(x_train['player_id'])
rating = pd.DataFrame({'player_id': players})
rating['game_strength'] = model.coef_[0][:len(players)]
rating = rating.sort_values(by='game_strength', ascending=False)
rating = pd.merge(rating, players_df, on="player_id").set_index('player_id')

In [16]:
rating.head(20)

,game_strength,name,patronymic,surname
player_id,,,,
5697,3.118317,Игорь,,Власенко
5701,3.043201,Дмитрий,Викторович,Власенков
5955,2.941970,Вадим,Сергеевич,Волотка
5791,2.940028,Владимир,,Войтехович
5703,2.908621,Евгений,Анатольевич,Власик
5936,2.887240,Евгения,,Володина
5012,2.882731,С.,А.,Варуха
6008,2.868204,Максим,,Воркунков
5980,2.854749,Станислав,Анатольевич,Волчёнков
